**Programmer:** python_scripts (Abhijith Warrier)

**PYTHON SCRIPT TO **_PREVENT MUTABLE DEFAULT BUGS USING dataclasses.field(default_factory=...)._** 🐍🛡️**

This script shows why using mutable objects (like list/dict) as defaults in dataclasses is dangerous and how to fix it using
default_factory. We’ll see the bug, the fix, and practical variations.

### 📦 Import Standard Library

In [2]:
from dataclasses import dataclass, field   # dataclass utilities and safe default factory
from typing import List, Dict              # type hints for clarity

### 📝 Snippet 1 — The Bug: Mutable Defaults Are Shared

Using a mutable object like [] as a default in a dataclass field causes all instances to share the same list.

In [10]:
from dataclasses import dataclass
from typing import List

try:
    @dataclass
    class BadTagStore:
        tags: List[str] = []   # ❌ BAD: raises ValueError in Python 3.11+

except ValueError as ve:
    print(f"ValueError - {str(ve)}")
# Running this in Python 3.11+ will throw:
# ValueError: mutable default <class 'list'> for field tags is not allowed: use default_factory

ValueError - mutable default <class 'list'> for field tags is not allowed: use default_factory


### 🛡️ Snippet 2 — The Fix: default_factory=list

field(default_factory=list) creates a fresh list per instance, preventing accidental sharing.

In [4]:
@dataclass                                       # safer dataclass
class GoodTagStore:                              # corrected class
    tags: List[str] = field(default_factory=list)  # ✅ GOOD: a new list each time

good1 = GoodTagStore()                           # first safe instance
good2 = GoodTagStore()                           # second safe instance
good1.tags.append("python")                      # mutate tags on first instance

print("Good store1:", good1.tags)                # ['python']
print("Good store2:", good2.tags)                # [] (independent)
print("Same object?", good1.tags is good2.tags)  # False

Good store1: ['python']
Good store2: []
Same object? False


### 🧰 Snippet 3 — Practical Variations

default_factory works not just for lists, but for dicts, sets, and even custom callables.

In [5]:
@dataclass
class ConfigStore:
    # dict gets a fresh copy per instance
    config: Dict[str, str] = field(default_factory=dict)

    # set gets a fresh copy per instance
    features: set = field(default_factory=set)

    # even a custom factory function
    def_list: List[str] = field(default_factory=lambda: ["default"])

c1 = ConfigStore()
c2 = ConfigStore()
c1.config["mode"] = "dark"

print("Config1:", c1.config)        # {'mode': 'dark'}
print("Config2:", c2.config)        # {} — independent

print("Features1:", c1.features)    # set()
print("Default list2:", c2.def_list) # ['default']

Config1: {'mode': 'dark'}
Config2: {}
Features1: set()
Default list2: ['default']


### ✅ One-liner Takeaway

Always use default_factory for mutable defaults in dataclasses — it prevents shared state bugs.